In [1]:
import pandas as pd

### Naive Bayes trial with first version of train data
(remove punctuation marks,
correct spacing between words,
split sentences,
classify part of speech)

In [2]:
train = pd.read_csv('./datas/train_ver1', index_col=[0])
train.tail()

,comments,bias_label,gender_label,hate_label,news_title,comment_pos,title_pos
7891,"힘내세요,응원합니다",none,False,none,"허지웅 허투루 넘길 말 없었다,솔직하게 드러냈던 속 사정","[('힘내세요', 'Verb'), (',', 'Punctuation'), ('응원'...","[('허지웅', 'Noun'), ('허투루', 'Noun'), ('넘길', 'Ver..."
7892,"힘내세요,삼가 고인의 명복을 빕니다",none,False,none,이혜경 오 캐롤 공연 중 남편 오정욱 부 고 오열 속 발인 종합,"[('힘내세요', 'Verb'), (',', 'Punctuation'), ('삼가'...","[('이혜경', 'Noun'), ('오', 'Noun'), ('캐롤', 'Noun'..."
7893,힘내세용 항상 응원합니닷,none,False,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,"[('힘내세용', 'Verb'), ('항상', 'Noun'), ('응원', 'Nou...","[('설경구', 'Noun'), ('송윤아', 'Noun'), ('아들', 'Nou..."
7894,"힘내 소연기로 답해요,나도 53살 인데 이런 일 저런 일 다 있더라구 요인격을 믿습...",none,False,none,SC 현장 연예인 인생 협박 유감 미소 잃은 최민수 보복운전 혐의 2차 공판 종합,"[('힘내', 'Verb'), ('소', 'Modifier'), ('연기', 'No...","[('SC', 'Alpha'), ('현장', 'Noun'), ('연예인', 'Nou..."
7895,힘들면 관뒀어야지 그게 현명한 거다,none,False,none,단독 스태프 사망 사고 서른 이지만 결국 오늘 촬영 취소,"[('힘들면', 'Adjective'), ('관', 'Noun'), ('뒀어야지',...","[('단독', 'Noun'), ('스태프', 'Noun'), ('사망', 'Noun..."


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7896 entries, 0 to 7895
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comments      7896 non-null   object
 1   bias_label    7896 non-null   object
 2   gender_label  7896 non-null   bool  
 3   hate_label    7896 non-null   object
 4   news_title    7896 non-null   object
 5   comment_pos   7896 non-null   object
 6   title_pos     7896 non-null   object
dtypes: bool(1), object(6)
memory usage: 439.5+ KB


In [4]:
a = train['comments'].to_list()
b = train['hate_label'].to_list()

In [5]:
train_list = list(zip(a, b))

In [6]:
from konlpy.tag import Okt
t = Okt()

In [8]:
train_docs = [(t.morphs(sentence[0]), sentence[1]) for sentence in train_list]

In [12]:
stopwords = pd.read_csv('https://bab2min.tistory.com/attachment/cfile2.uf@241D6F475873C2B1010DEA.txt', sep='\t', header=None, names=['형태','품사','비율'])
stopwords_list = stopwords['형태'].tolist()

def process_text(text):
    clean_words = [word for word in text if word not in stopwords_list]
    return clean_words

In [16]:
train_docs = process_text(train_docs)

In [17]:
all_words = set([i for d in train_docs for i in d[0]])

In [18]:
def term_exists(doc):
    return {word: (word in set(doc)) for word in all_words}

In [19]:
train_xy = [(term_exists(d), c) for d,c in train_docs]

In [20]:
import nltk

classifier = nltk.NaiveBayesClassifier.train(train_xy)
classifier.show_most_informative_features()

Most Informative Features
                      돼지 = True             hate : none   =     44.4 : 1.0
                       쳐 = True             hate : none   =     40.5 : 1.0
                      생긴 = True             hate : none   =     37.1 : 1.0
                      재앙 = True             hate : offens =     36.2 : 1.0
                      새끼 = True             hate : none   =     31.0 : 1.0
                     OOO = True             hate : none   =     24.9 : 1.0
                       ㅈ = True             hate : none   =     18.8 : 1.0
                      쟁이 = True             hate : offens =     18.7 : 1.0
                      페미 = True             hate : offens =     18.7 : 1.0
                      지랄 = True             hate : none   =     17.6 : 1.0


In [21]:
dev = pd.read_csv('./datas/dev.hate.csv')
dev.rename(columns={'label': 'hate_label'}, inplace=True)
dev.tail()

,comments,hate_label
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate
470,남자가 잘못한거라면... 반성도 없다면...나였다면 ... 여자처럼 아주 못되게 할...,none


In [22]:
test_sentence = '여자인생 망칠 일 있나 ㅋㅋ'
classifier.classify(term_exists(t.morphs(test_sentence)))

'none'

In [23]:
pred = []
for sentence in dev['comments']:
    pred.append(classifier.classify(term_exists(t.morphs(sentence))))

In [24]:
from sklearn.metrics import f1_score, accuracy_score, recall_score
f1_score(pred, dev['hate_label'], average='macro')

0.509707870249717

### Naive Bayes trial with second version of train data
(remove punctuation marks,
classify part of speech)

In [25]:
train_2 = pd.read_csv('./datas/train_ver2', index_col=[0])
train_2.tail()

,comments,bias_label,gender_label,hate_label,news_title,comment_pos,title_pos
7891,힘내세요 응원합니다,none,False,none,허지웅 허투루 넘길 말 없었다 솔직하게 드러냈던 속사정,"[('힘내세요', 'Verb'), ('응원', 'Noun'), ('합니다', 'Ve...","[('허지웅', 'Noun'), ('허투루', 'Noun'), ('넘길', 'Ver..."
7892,힘내세요삼가 고인의 명복을 빕니다,none,False,none,이혜경 오 캐롤 공연 중 남편 오정욱 부고 오열 속 발인 종합,"[('힘내세요', 'Verb'), ('삼가', 'Verb'), ('고인', 'Nou...","[('이혜경', 'Noun'), ('오', 'Noun'), ('캐롤', 'Noun'..."
7893,힘내세용 항상 응원합니닷,none,False,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,"[('힘내세용', 'Verb'), ('항상', 'Noun'), ('응원', 'Nou...","[('설경구', 'Noun'), ('송윤아', 'Noun'), ('아들', 'Nou..."
7894,힘내소연기로 답해요나도 53살 인데 이런일 저런일 다 있더라구요인격을 믿습니다홨팅,none,False,none,SC현장연예인 인생 협박 유감 미소잃은 최민수 보복운전 혐의 2차 공판종합,"[('힘내소', 'Verb'), ('연기', 'Noun'), ('로', 'Josa'...","[('SC', 'Alpha'), ('현장', 'Noun'), ('연예인', 'Nou..."
7895,힘들면 관뒀어야지 그게 현명한거다,none,False,none,단독스태프 사망사고 서른이지만 결국 오늘 촬영 취소,"[('힘들면', 'Adjective'), ('관', 'Noun'), ('뒀어야지',...","[('단독', 'Noun'), ('스태프', 'Noun'), ('사망', 'Noun..."


In [26]:
c = train_2['comments'].to_list()
d = train_2['hate_label'].to_list()
train2_list = list(zip(c, d))

In [27]:
train2_docs = [(t.morphs(sentence[0]), sentence[1]) for sentence in train2_list]
train2_docs = process_text(train2_docs)
all_words = set([i for d in train2_docs for i in d[0]])
train2_xy = [(term_exists(d), c) for d,c in train2_docs]

classifier = nltk.NaiveBayesClassifier.train(train2_xy)
classifier.show_most_informative_features()

Most Informative Features
                       년 = True             hate : none   =     37.8 : 1.0
                      재앙 = True             hate : offens =     36.2 : 1.0
                      돼지 = True             hate : none   =     31.0 : 1.0
                      새끼 = True             hate : none   =     29.8 : 1.0
                       쳐 = True             hate : none   =     26.3 : 1.0
                     OOO = True             hate : none   =     24.9 : 1.0
                      페미 = True             hate : offens =     24.8 : 1.0
                     개돼지 = True             hate : offens =     24.0 : 1.0
                      미투 = True             hate : none   =     20.1 : 1.0
                       ㅈ = True             hate : none   =     18.8 : 1.0


In [28]:
pred2 = []
for sentence in dev['comments']:
    pred2.append(classifier.classify(term_exists(t.morphs(sentence))))

f1_score(pred2, dev['hate_label'], average='macro')

0.5254201266795134